## SANDBOX 

In [ ]:
import elevation
!eio clip -o Shasta-30m-DEM.tif --bounds -122.6 41.15 -121.9 41.6

## Segmenting the Trails - Killington / Stratton 

In [ ]:
def Seg_Terrain(mapIM_0, bShow=0):
    """Segment ski trails from google terrain view"""
    
    npMap = np.array(mapIM_0)[...,:3]/255
    vectors = np.array([[.2,1,0],[1,0,0],[0,0,1],[0,0,0],[.5,.5,.5]])
    
    npDoT = np.full(npMap.shape[0:2] + (vectors.shape[0],), -1,"f4")

    for i in range(vectors.shape[0]):

        #Get Distance to color
        npTemp = np.linalg.norm(npMap - [[vectors[i]]], axis=2)
        npTemp = -npTemp

        npTemp[npTemp<npTemp.mean()] = npTemp.mean() 
        # Highpass filtering to get lines 
        npTemp = npTemp - gaussian_filter(npTemp, sigma=20)
        npTemp[npTemp<0] = 0
        
        #Force intensity into set distribution 
        topDev = np.median(npTemp)+3*npTemp.std()
        botDev = np.median(npTemp)-3*npTemp.std()        
        npTemp = npTemp.clip(botDev,topDev)
        
        npTemp = (npTemp - npTemp.min()) / (npTemp.max() - npTemp.min())
        
        #Collect image maps 
        npDoT[:,:,i] = npTemp
        
    #Thresholding and morphology
    segIM = np.zeros((np.array(npMap).shape[0:2]))-1
    for i in range(vectors.shape[0]):
        bSeg = npDoT[...,i]>0.75
        bSeg = morphology.remove_small_objects(bSeg,20)
        segIM[bSeg] = i+1
    
    ## Show a troubleshooting figure out of the segmentation
    if bShow:
        X1,X2,Y1,Y2 = 0,1000,0,1600
        crop_rectangle = (Y1, X1, Y2, X2)

        labIM = skicolor.label2rgb(segIM, colors = vectors, bg_color = (.3,0.3,0.3))
        if 0:
            f, axarr = plt.subplots(1,2,figsize=(40, 20))

            axarr[0].imshow(mapIM_0.crop(crop_rectangle))
            axarr[1].imshow(labIM[X1:X2,Y1:Y2])
        else:
            f, ax = plt.subplots(2,3,figsize=(20, 10))
            ax[0,0].imshow(mapIM_0.crop(crop_rectangle))
            ax[0,1].imshow(labIM[X1:X2,Y1:Y2])
            ax[0,2].imshow(npDoT[X1:X2,Y1:Y2,0])
            ax[1,0].imshow(npDoT[X1:X2,Y1:Y2,1])
            ax[1,1].imshow(npDoT[X1:X2,Y1:Y2,2])
            ax[1,2].imshow(npDoT[X1:X2,Y1:Y2,3])
        plt.tight_layout()
    return segIM

def Seg_Trails_Sat(Sat_IM, bShow=0):
    """Segment ski trails from google Satelite view"""
    npSat = np.array(Sat_IM)[...,:3]/255
    npSat = np.linalg.norm(npSat, axis=2)

    topDev = np.median(npSat)+2*npSat.std()
    botDev = np.median(npSat)-2*npSat.std()
    npSat = npSat.clip(botDev,topDev)

    npSat = npSat - gaussian_filter(npSat, sigma=31)
    npSat = ndimage.median_filter(npSat, size=3)
    npSat[npSat<0] = 0
    thresh = np.median(npSat)+npSat.std()

    seg_Sat = npSat > thresh
    seg_Sat = morphology.remove_small_objects(seg_Sat,100)
    
    ## Show a troubleshooting figure out of the segmentation
    if bShow:
        segShow = np.array(Sat_IM)[...,:3]/255 
        segShow[seg_Sat,:] = 0.8,0,0

        f, ax = plt.subplots(2,2,figsize=(40, 20))
        ax[0,0].imshow(Sat_IM,cmap = 'jet');
        ax[0,1].imshow(npSat,cmap = 'jet');

        ax[1,0].imshow(segShow,cmap = 'jet');
        ax[1,1].hist(npSat[npSat>0].ravel(), bins = 256);
        ax[1,1].plot([thresh,thresh],[0, 5000])
    
    return seg_Sat   

def pad_Image_square(IM):
    padSize =  np.max(IM.shape) - IM.shape
    padSize = np.array([padSize*0 , padSize]).T
    IM_out = np.pad(IM, padSize, "constant", constant_values=0)
    
    return IM_out 

def resize_image_to_match(im1,im2,scale=1):

    out_size = np.round(np.maximum(im1.shape, im2.shape)*scale)
    im1_out = resize(im1,out_size)
    im2_out = resize(im2,out_size)
    
    return im1_out, im2_out

In [ ]:
plt.close('all')
mapIM_0 = Image.open("../Satelite/Killington _43.6107026_ -72.796019,4995_terrian.JPG")
#mapIM_0 = Image.open("../Satelite/Killington _43.6107026_ -72.796019,4995.JPG")
seg_Ter = Seg_Terrain(mapIM_0,0)

Sat_IM = Image.open("../Satelite/Killington_Sat.jpg")
seg_Sat = Seg_Trails_Sat(Sat_IM,0)


In [ ]:
plt.close('all')

mapIM_0 = Image.open("../Satelite/Killington _43.6107026_ -72.796019,4995_terrian.JPG")
#mapIM_0 = Image.open("../Satelite/Killington _43.6107026_ -72.796019,4995.JPG")

bExport = 0

Coor = pd.read_excel('CoOrLists.xls')
Coor.at[10,["X1","X2","Y1","Y2"]] = [1610,1720,2580,2690]

DEM = GetDem(Root, Coor,10)
DEM = resize(DEM, np.round(np.array(DEM.shape) * 2000 / np.amax(DEM.shape)))

segIM = Seg_Terrain(mapIM_0)
## Add Annotations

#segIM = resize(segIM, DEM.shape)

demPts = np.array([[660,1190],[680,930],[580,730],[1010,1110]])
satPts = np.array([[425,640],[434,327],[365,80],[750,560]])

model_robust, inliers = ransac((satPts,demPts), SimilarityTransform,
                               min_samples=10, residual_threshold=1, 
                               max_trials=300)

segIM_reg = warp(np.array(segIM), model_robust.inverse, preserve_range=True,
               output_shape=DEM.shape, cval=-1)


## Plot Registration 

#DEM[segIM_reg>-1] = 0
DEM[segIM_reg>-1] = np.min(DEM)

%matplotlib qt

f, ax = plt.subplots(1,2,figsize=(40, 20))
ax[0].imshow(DEM,cmap = 'jet');
ax[0].plot(demPts[:,0],demPts[:,1],'ro')
ax[1].imshow(mapIM_0);
ax[1].plot(satPts[:,0],satPts[:,1],'ro')

f, ax = plt.subplots(1,1,figsize=(40, 20))
ax.imshow(DEM,cmap = 'jet');

In [ ]:
def OrbRegistation(image1,image2, bShow=0):
    
    orb = ORB(n_keypoints=500, fast_threshold=0.05)
    orb.detect_and_extract(image1)
    keypoints1 = orb.keypoints
    descriptors1 = orb.descriptors

    orb.detect_and_extract(image2)
    keypoints2 = orb.keypoints
    descriptors2 = orb.descriptors

    matches12 = match_descriptors(descriptors1, descriptors2, cross_check=True)

    src = keypoints2[matches12[:, 1]][:, ::-1]
    dst = keypoints1[matches12[:, 0]][:, ::-1]

    model_robust, inliers = ransac((src, dst), SimilarityTransform,
                                   min_samples=10, residual_threshold=1, 
                                   max_trials=300)
    
    image2_reg = warp(np.array(image2), model_robust.inverse, preserve_range=True,
               output_shape=DEM.shape, cval=-1)
    
    
    if bShow==1:
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        plot_matches(ax, image1, image2, keypoints1, keypoints2, matches12[inliers])
        ax.axis('off')
        plt.show()
    elif bShow==2:    
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        plot_matches(ax, image1, image2, keypoints1, keypoints2, matches12, only_matches=True)
        plt.axis('off')
        plt.show()
        
    return image2_reg

In [ ]:
from skimage.feature import ORB, match_descriptors, plot_matches

image1 = Image.open("../Satelite/Killington _43.6107026_ -72.796019,4995_terrian.JPG")
image2 = Image.open("../Satelite/Killington _43.6107026_ -72.796019,4995.JPG")

image1 = np.array(Seg_Terrain(image1)>-1)
image2 = np.array(Seg_Terrain(image2)>-1)

OrbRegistation(image1,image2)


## Registration 

In [ ]:
def showReg(im,im2,im2_reg):
    plt.close('all')
    f, ax = plt.subplots(2,2,figsize=(40, 20))
    ax[0,0].imshow(im)
    ax[0,1].imshow(im2)

    ax[1,0].imshow(im2_reg)

    layers = im2_reg - 2*im
    ax[1,1].imshow(layers)

    plt.tight_layout()

In [ ]:
import imreg_dft
from skimage.transform import ProjectiveTransform, SimilarityTransform, warp
from skimage.feature import register_translation
from skimage.feature import ORB, match_descriptors, plot_matches

## Load & Seg Data 
im_terrian = Image.open("../Satelite/Killington _43.6107026_ -72.796019,4995_terrian.JPG")
seg_ter = Seg_Terrain(im_terrian)

seg_Sat = Image.open("../Satelite/Killington_Sat.jpg")
seg_Sat = Seg_Trails_Sat(seg_Sat)


## CLean Data to make easier to register 
seg_ter = seg_ter[:800,300:1400]
seg_ter = np.logical_and(seg_ter > -1 , seg_ter < 5)
seg_ter = morphology.binary_dilation(seg_ter,morphology.disk(3))
seg_ter = pad_Image_square(seg_ter*1)

#seg_Sat = morphology.binary_dilation(seg_Sat,morphology.disk(3))
seg_Sat = seg_Sat *1
#seg_Sat = (seg_Sat * 2)-1
seg_Sat = pad_Image_square(seg_Sat)

seg_ter, seg_Sat = resize_image_to_match(seg_ter, seg_Sat)
seg_ter2 = gaussian_filter(seg_ter, sigma=20) + 0.1*seg_ter
seg_Sat2 = gaussian_filter(seg_Sat, sigma=20) + 0.1*seg_Sat

constraints = {"angle": [0,1]}

X = imreg_dft.imreg.similarity(seg_ter2, seg_Sat2, numiter=10,constraints=constraints)

seg_Sat_reg = imreg_dft.imreg.transform_img(seg_Sat, tvec=X["tvec"])

print(X["success"])

if X["success"] > 0.7
    showReg(seg_ter,seg_Sat,seg_Sat_reg)
#imreg_dft.imreg.translation(segIM, seg_Sat)
#imreg_dft.imreg.imshow(im0, im1, im2)